In [21]:
import gymnasium as gym
import torch.nn as nn
import random
import torch

In [22]:
env=gym.make("CartPole-v0")  

c:\Python\Lib\site-packages\gymnasium\envs\registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [23]:
"""Random agent"""
rewards=0

for _ in range(100):
    current_trajectory=[]
    state=env.reset()[0]
    terminal=False

    while terminal==False:
        

        action=random.randint(0,1)

        next_state,reward,terminal,truncated,_=env.step(action)

        rewards+=reward
        if truncated==True:
            break

        state=next_state
    
print(rewards/100)

22.74


In [24]:
class policyNetwork:

    def __init__(self):
        self.L=nn.Sequential(
            nn.Linear(4,32),
            nn.ReLU(),
            nn.Linear(32,32),
            nn.ReLU(),
            nn.Linear(32,2),
            nn.Softmax()
        )
    
    def forward(self,x):
        return self.L(x)   

In [25]:
max_iters=1500
K=32
gamma=0.9
advantage_normalization=True
reward_to_go=True
model=policyNetwork()
optimizer=torch.optim.Adam(params=model.L.parameters(),lr=0.001)

for _ in range(max_iters):

    trajectories=[]
    for _ in range(K):
        current_trajectory=[]
        state=env.reset()[0]
        terminal=False

        while terminal==False:
            
            weights=model.forward(torch.tensor(state))
  
            actions=[0,1]
            action=random.choices(actions,list(weights.detach()))

            next_state,reward,terminal,truncated,_=env.step(action[0])


            current_trajectory.append((state,action,reward))
            
            if truncated==True:
                break

            state=next_state

        trajectories.append(current_trajectory)

    if reward_to_go==True:
        A=[]
        for i in range(K):
            G=[]

            for t in range(len(trajectories[i])):
                sum=0

                for j in range(t,len(trajectories[i])):
                        sum+=(gamma**(j-t) * trajectories[i][j][2])
                    
                G.append([sum,trajectories[i][t][0],trajectories[i][t][1]])
        
            A.append(G)
    else:
        A=[]
        for i in range(K):
            G=[]
            sum=0
            for t in range(len(trajectories[i])): 
                sum+=(gamma**(t) * trajectories[i][t][2])
            G.append([sum,trajectories[i][0][0],trajectories[i][0][1]])
        
            A.append(G)

    
    if advantage_normalization==True:
        #calculating A_t
        for i in range(len(A)):
            sum=0
            for j in range(len(A[i])):
                sum+=A[i][j][0]
            
            sum=sum/len(A[i])

            for j in range(len(A[i])):
                A[i][j][0]-=sum
    
    J=0

    for i in range(K):

        for j in range(len(A[i])):

            a,state,action=A[i][j]

            J+=(torch.log(model.forward(torch.tensor(state))[action])*a)

    J=J/K

    optimizer.zero_grad()
    J.backward()
    optimizer.step()

c:\Python\Lib\site-packages\torch\nn\modules\container.py:217: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


In [26]:

rewards=0

for _ in range(100):
    current_trajectory=[]
    state=env.reset()[0]
    terminal=False

    while terminal==False:
        
        weights=model.forward(torch.tensor(state))

        action=torch.argmax(weights)


        next_state,reward,terminal,truncated,_=env.step(int(action.detach()))
        if truncated==True:
                break

        rewards+=reward

        state=next_state
    
print(rewards/100)

9.03
